# Gait generation

The state and control vectors $\textbf{x}$ and $\textbf{u}$ are defined as follows:

$$
\begin{equation*}
\textbf{X} = \begin{bmatrix}
    x & y & z & \dot{x} &\dot{y} &\dot{z} 
    \end{bmatrix}^T
\end{equation*}
$$

$$
\begin{equation*}
    \textbf{u} = \begin{bmatrix}
    x_z & y_z & F_z
    \end{bmatrix}^T
\end{equation*}
$$


The dynamic equation for the system will be 

$$
\begin{equation*}
    \begin{bmatrix}
    \dot{X}  
    \end{bmatrix} 
    =
    \begin{bmatrix}
    \dot{x} \\ 
    \dot{y} \\ 
    \dot{z} \\ 
    \frac{(x-x_z)F_z}{mz} \\ 
    \frac{(y-y_z)F_z}{mz} \\ 
    \frac{F_z}{m}-g
    \end{bmatrix}
\end{equation*}
$$


where $\textbf{m}$ and $\textbf{g}$ are mass and the acceleration of gravity. In the following case, I will set $\textbf{m}$ to be 1.






In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

In [4]:
import numpy as np
import theano.tensor as T
import matplotlib.pyplot as plt

In [5]:
from ilqr import iLQR
from ilqr.cost import PathQRCost
from ilqr.dynamics import AutoDiffDynamics

In [6]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    print("iteration", iteration_count, info, J_opt)

In [7]:
x_inputs = [
    T.dscalar("x"),
    T.dscalar("y"),
    T.dscalar("z"),
    T.dscalar("x_dot"),
    T.dscalar("y_dot"),
    T.dscalar("z_dot"),
]

u_inputs = [
    T.dscalar("x_zmp"),
    T.dscalar("y_zmp"),
    T.dscalar("F_z"),
]



dt = 0.01  # Discrete time step.
g = 9.81  # g.

'''
# nonlinear dynamics.
def nonldyn(x_inputs, u_inputs):
    x_inputs_dot[0]=x_inputs[3],
    x_inputs_dot[1]=x_inputs[4],
    x_inputs_dot[2]=x_inputs[5],
    x_inputs_dot[3]=(x_inputs[0]-u[0])*u[2]/x_inputs[2],
    x_inputs_dot[4]=(x_inputs[1]-u[1])*u[2]/x_inputs[2],
    x_inputs_dot[5]=u[2]-g,
    return x_inputs_dot

# discritize with RK4
x_inputs_dot1=nonldyn(x_inputs,u_inputs)
x_inputs_dot2=nonldyn(x_inputs+.5*dt*x_inputs_dot1,u_inputs)
x_inputs_dot3=nonldyn(x_inputs+.5*dt*x_inputs_dot2,u_inputs)
x_inputs_dot4=nonldyn(x_inputs+dt*x_inputs_dot3,u_inputs)

f = x_inputs+(dt/6)*(x_input_dot1+2*x_input_dot2+2*x_input_dot3+x_input_dot4)
'''


# Discrete dynamics model definition.
f = T.stack([
    x_inputs[0] + x_inputs[3] * dt,
    x_inputs[1] + x_inputs[4] * dt,
    x_inputs[2] + x_inputs[5] * dt,
    x_inputs[3] + (x_inputs[0]-u_inputs[0])*u_inputs[2]/x_inputs[2] * dt,
    x_inputs[4] + (x_inputs[1]-u_inputs[1])*u_inputs[2]/x_inputs[2] * dt,
    x_inputs[5] + (u_inputs[2]-g) * dt,
])


dynamics = AutoDiffDynamics(f, x_inputs, u_inputs)

In [8]:
T = 100
state_size = 6
action_size = 3

Q = np.eye(state_size)
R = np.eye(action_size)

x_path = np.zeros((T+1, state_size))
u_path = np.zeros((T, action_size))


cost = PathQRCost(Q, R, x_path, u_path=u_path)


In [11]:
N = 200  # Number of time steps in trajectory.
x0 = np.array([0, 0, 0, 0, 0, 0])  # Initial state.

# Random initial action path.
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))

In [12]:
J_hist = []
ilqr = iLQR(dynamics, cost, N)
xs, us = ilqr.fit(x0, us_init, on_iteration=on_iteration)

iteration 0 failed nan
iteration 1 failed nan
iteration 2 failed nan
iteration 3 failed nan
iteration 4 failed nan
iteration 5 failed nan


C:\SciSoft\Anaconda2\lib\site-packages\ilqr-0.1.0-py2.7.egg\ilqr\controller.py:134: UserWarning: SVD did not converge
C:\SciSoft\Anaconda2\lib\site-packages\ilqr-0.1.0-py2.7.egg\ilqr\controller.py:141: UserWarning: exceeded max regularization term


UnboundLocalError: local variable 'k' referenced before assignment